In [1]:
{-# LANGUAGE RankNTypes #-}

import AdventOfCode
import qualified Data.Vector.Unboxed as V
import qualified Data.Vector.Unboxed.Mutable as M
import Data.List
import Data.Maybe
import Control.Monad.ST
import Data.STRef
import Data.Function (fix)

input <- V.fromList . map readInt . words <$> dayString 6

In [2]:
redistribute :: V.Vector Int -> V.Vector Int
redistribute v = withThawed v $ \mut -> do
    let i = maxIndex v
    blocks <- newSTRef =<< empty mut i
    pointer <- newSTRef (i+1)
    go mut pointer blocks
    return ()
    where
        go :: M.STVector s Int -> STRef s Int -> STRef s Int -> ST s (M.STVector s Int)
        go vector index blocks = do
            count <- readSTRef blocks
            i <- (`mod` M.length vector) <$> readSTRef index
            if count == 0
                then return vector
                else do
                    M.modify vector (+1) i
                    writeSTRef index (i+1)
                    writeSTRef blocks (count-1)
                    go vector index blocks
        empty :: M.STVector s Int -> Int -> ST s Int
        empty mut i = do
            blocks <- M.read mut i
            M.write mut i 0
            return blocks
        maxIndex :: V.Vector Int -> Int
        maxIndex v = fromJust $ V.elemIndex (V.maximum v) v
        withThawed :: (V.Unbox a) => V.Vector a -> (forall s. M.STVector s a -> ST s ()) -> V.Vector a
        withThawed vector f = runST $ do
            mut <- V.thaw vector
            f mut
            V.freeze mut

In [3]:
main  = fix $ \rec moves i -> let
    (before, after) = splitAt i moves
    current = head after
    in if current `elem` before then (i, i - fromJust (elemIndex current before)) else rec moves (i+1)

main (iterate redistribute input) 0

(12841,8038)